# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante:
Deiry Sofia Navas Muriel
#### Segundo Integrante:
David Alejandro Marin alzate

In [7]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from __future__ import division


#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Para este ejercicio usaremos la base de datos sobre calidad del aire, que ha sido usada en laboratorios previos, pero en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [8]:
#cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter

#Esta es la base de datos AirQuality del UCI Machine Learning Repository. En la siguiente URL se encuentra toda
#la descripción de la base de datos y la contextualización del problema.
#https://archive.ics.uci.edu/ml/datasets/Air+Quality#

X = db[:,0:11]
Y = db[:,11:13]

In [9]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

## Ejercicio 1

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [10]:
from numpy import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
#Validamos el modelo
def main(hidden = (28,)):
    Folds = 4
    random.seed(19680801)
    ErrorY1 = np.zeros(Folds)
    ErrorY2 = np.zeros(Folds)
    ErrorT = np.zeros(Folds)
    ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
    j = 0
    for train, test in ss.split(X):
        Xtrain = X[train,:]
        Ytrain = Y[train,:]
        Xtest = X[test,:]
        Ytest = Y[test,:]

        #Normalizamos los datos
        media = np.mean(Xtrain,axis=0)
        desvia = np.std(Xtrain,axis=0)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp = MLPRegressor(hidden_layer_sizes= hidden, activation='tanh', max_iter= 500)
        mlp.fit(Xtrain,Ytrain)
        #Use para el modelo para hacer predicciones sobre el conjunto Xtest
        Yest = mlp.predict(Xtest)

        #Mida el error MAPE para cada una de las dos salidas
        ErrorY1[j] = MAPE(Yest[:,0], Ytest[:,0])
        ErrorY2[j] = MAPE(Yest[:,1], Ytest[:,1])
        ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
        j += 1
        
    mean1 = round(np.mean(ErrorY1),5)
    std1 = round(np.std(np.std(ErrorY1)),5)
    
    mean2 = round(np.mean(ErrorY2),5)
    std2 = round(np.std(ErrorY2),5)
    
    meanT = round(np.mean(ErrorT),5)
    stdT = round(np.std(ErrorT),5)

    return mean1,std1,mean2,std2,meanT,stdT

mean1,std1,mean2,std2,meanT,stdT = main(hidden= (28,))

print('MAPE salida 1 = ' + str(mean1) + '+-' + str(std1))
print('MAPE salida 2 = ' + str(mean2) + '+-' + str(std2))
print('MAPE total = '    + str(meanT) + '+-' + str(stdT))
    


/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MAPE salida 1 = 0.07725+-0.0
MAPE salida 2 = 1.73297+-0.06943
MAPE total = 0.90511+-0.03471


/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [11]:
num_layers = [1,1,1,1,1,2,2,2,2,2]
num_neurons_per_layer = [20,24,28,32,36,20,24,28,32,36]
mape_1 = np.zeros(10)
mape_ic_1 = np.zeros(10)
mape_2 = np.zeros(10)
mape_ic_2 = np.zeros(10)

for i in range(0,10):
    if(num_layers[i] == 1):
        mape_1[i],mape_ic_1[i],mape_2[i],mape_ic_2[i],meanT,stdT = main(hidden= (num_neurons_per_layer[i]))
    elif(num_layers[i] == 2):
        mape_1[i],mape_ic_1[i],mape_2[i],mape_ic_2[i],meanT,stdT = main(hidden= (num_neurons_per_layer[i],num_neurons_per_layer[i]))
    print(i)
    print('MAPE salida 1 = ' + str(mape_1[i]) + '+-' + str(mape_ic_1[i]))
    print('MAPE salida 2 = ' + str(mape_2[i]) + '+-' + str(mape_ic_2[i]))
    print('MAPE total = '    + str(meanT) + '+-' + str(stdT))

/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  %

0
MAPE salida 1 = 0.08623+-0.0
MAPE salida 2 = 2.39858+-0.15855
MAPE total = 1.24241+-0.07903


/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1
MAPE salida 1 = 0.08204+-0.0
MAPE salida 2 = 2.07043+-0.11873
MAPE total = 1.07623+-0.05892


/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  %

...

3
MAPE salida 1 = 0.07699+-0.0
MAPE salida 2 = 1.51588+-0.05246
MAPE total = 0.79644+-0.02698


/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


4
MAPE salida 1 = 0.0757+-0.0
MAPE salida 2 = 1.46091+-0.07528
MAPE total = 0.7683+-0.03798


/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/alejandro/py3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  %

5
MAPE salida 1 = 0.07142+-0.0
MAPE salida 2 = 1.2208+-0.09974
MAPE total = 0.64611+-0.05068
6
MAPE salida 1 = 0.06621+-0.0
MAPE salida 2 = 1.10994+-0.08679
MAPE total = 0.58807+-0.04318
7
MAPE salida 1 = 0.06788+-0.0
MAPE salida 2 = 1.00287+-0.03377
MAPE total = 0.53538+-0.01678
8
MAPE salida 1 = 0.06722+-0.0
MAPE salida 2 = 1.09435+-0.1408
MAPE total = 0.58079+-0.07162
9
MAPE salida 1 = 0.06601+-0.0
MAPE salida 2 = 1.02205+-0.07211
MAPE total = 0.54403+-0.03573


## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [13]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["MAPE salida 1"] = mape_1
df_types["IC MAPE salida 1"] = mape_ic_1
df_types["MAPE salida 2"] = mape_2
df_types["IC MAPE salida 2"] = mape_ic_2
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [14]:
qgrid_widget.get_changed_df()

MAPE salida 1  IC MAPE salida 1  \
N. de capas ocultas Neuronas por capa                                    
1                   20                       0.08623               0.0   
                    24                       0.08204               0.0   
                    28                       0.07725               0.0   
                    32                       0.07699               0.0   
                    36                       0.07570               0.0   
2                   20                       0.07142               0.0   
                    24                       0.06621               0.0   
                    28                       0.06788               0.0   
                    32                       0.06722               0.0   
                    36                       0.06601               0.0   

                                       MAPE salida 2  IC MAPE salida 2  
N. de capas ocultas Neuronas por capa                                   
1                   20                       2.39858           0.15855  
                    24                       2.07043           0.11873  
                    28                       1.73297           0.06943  
                    32                       1.51588           0.05246  
                    36                       1.46091           0.07528  
2                   20                       1.22080           0.09974  
                    24                       1.10994           0.08679  
                    28                       1.00287           0.03377  
                    32                       1.09435           0.14080  
                    36                       1.02205           0.07211

<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

En este caso la red neuronal para regresión utliza la funcion de activación "identity", en realidad no corresponde a ninguna funcion de activación sino es el mismo valor de entrada $f(x)=x$

## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación de dígitos escritos a mano. Usaremos únicamente 4 de las 10 clases disponibles. Los datos fueron preprocesados para reducir el número de características. La técnica usada será analizada más adelante en el curso.

In [16]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
def mainClassifier(hidden = (24)):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp = MLPClassifier(activation='tanh', hidden_layer_sizes=hidden, max_iter=500 )
        mlp.fit(Xtrain,Ytrain)

        #Validación con las muestras de entrenamiento
        Ytrain_pred = mlp.predict(Xtrain)

        #Validación con las muestras de test    
        Yest = mlp.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1
    
    mean = round(np.mean(EficienciaTrain),5)
    std = round(np.std(EficienciaTrain),5)
    
    meanVal = round(np.mean(EficienciaVal),5)
    stdVal = round(np.std(EficienciaVal),5)
    
    return mean,std,meanVal,stdVal

#mean,std,meanVal,stdVal,mainClassifier()
#print('Eficiencia durante el entrenamiento = ' + str(mean) + '+-' + str(std))
#print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

In [34]:
num_layers = [1,1,1,1,1,2,2,2,2,2]
num_neurons_per_layer = [20,24,28,32,36,20,24,28,32,36]
mean = np.zeros(10)
std = np.zeros(10)
meanVal = np.zeros(10)
stdVal = np.zeros(10)

for i in range(0,10):
    if(num_layers[i] == 1):
        mean[i], std[i], meanVal[i], stdVal[i]= mainClassifier(hidden= (num_neurons_per_layer[i]))
    elif(num_layers[i] == 2):
        mean[i], std[i], meanVal[i], stdVal[i]= mainClassifier(hidden= (num_neurons_per_layer[i],num_neurons_per_layer[i]))
    print('Eficiencia durante el entrenamiento = ' + str(mean[i]) + '+-' + str(std[i]))
    print('Eficiencia durante la validación = '    + str(meanVal[i]) + '+-' + str(stdVal[i]))

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.96379+-0.02677
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9623+-0.03692
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.96378+-0.02468
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.95965+-0.02931
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.95537+-0.03395
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.96936+-0.02252
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.95265+-0.03747
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.95124+-0.03376
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.95686+-0.03491
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.96105+-0.01971


## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [35]:
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["Eficiencia en validacion"] = meanVal
df_types["Intervalo de confianza"] = stdVal
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [36]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  \
N. de capas ocultas Neuronas por capa                             
1                   20                                  0.96379   
                    24                                  0.96230   
                    28                                  0.96378   
                    32                                  0.95965   
                    36                                  0.95537   
2                   20                                  0.96936   
                    24                                  0.95265   
                    28                                  0.95124   
                    32                                  0.95686   
                    36                                  0.96105   

                                       Intervalo de confianza  
N. de capas ocultas Neuronas por capa                          
1                   20                                0.02677  
                    24                                0.03692  
                    28                                0.02468  
                    32                                0.02931  
                    36                                0.03395  
2                   20                                0.02252  
                    24                                0.03747  
                    28                                0.03376  
                    32                                0.03491  
                    36                                0.01971

<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

En la capa de la salida la función de activación es "softmax", se utliza el atributo del MLPRegressor out_activation_

4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Porqué debe tener ese número?

Hay 4 neuronas en la capa de salida que corresponde a las 4 clases del problema de clasificación, se utiliza el atributo  n_outputs_